# 근거리 행정동 시각화하기

In [3]:
# !pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 94.2 MB/s eta 0:00:00


In [8]:
import os

import numpy as np
import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.rc('font', family='Malgun Gothic') # 한글 폰트 적용
mpl.rcParams["axes.unicode_minus"] = False # 마이너스 기호 깨짐 방지
plt.rcParams["figure.figsize"] = (12, 8) # 차트 사이즈 설정

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


---

# 1. 데이터 불러오기

In [9]:
os.getcwd()

'/content'

In [7]:
geo_master = gpd.read_file('../../data/서울생활이동/gis-master/move-emd-geo-master.shp')
display(geo_master.head())
display(geo_master.shape)

ERROR:fiona._env:../../data/서울생활이동/gis-master/move-emd-geo-master.shp: No such file or directory


DriverError: ignored

# 2. 근거리 행정동 지도 시각화

In [ ]:
def show_geo(gdf, column):
    # 지도 시각화
    # fig = plt.figure()
    # plt.figure(figsize=(20, 20))
    gdf.plot(column=column,
             legend=True,
             cmap='Blues',
             edgecolor='k',
             legend_kwds={'label': '명'})
    plt.axis('off')
    plt.tight_layout()

    # plt.savefig('예제1.png')
    plt.show()

In [ ]:
rows = []
for _, _, emd_cd1, emd_nm1, geo1 in geo_master.values:
    for _, _, emd_cd2, emd_nm2, geo2 in geo_master.values:
        rows.append((emd_cd1, emd_nm1, emd_cd2, emd_nm2, geo1.distance(geo2)))

emd_dists = pd.DataFrame(rows, columns=['emd_cd1', 'emd_nm1', 'emd_cd2', 'emd_nm2', 'distance'])
emd_dists.head()

In [ ]:
geo_master.shape, emd_dists.shape

In [ ]:
geo = geo_master.copy()
geo.insert(3, 'value', 0.0)

In [ ]:
geo.head()

In [ ]:
geo['value'] = 0.0

# 랜덤 타겟 행정동
target_cd, target_nm = geo_master.sample().loc[:, ['emd_cd', 'emd_nm']].values[0]

# 타겟 행정동으로부터 근거리 행정동 nears
nears = emd_dists.loc[(emd_dists['emd_cd1'] == target_cd) & emd_dists['distance'].between(0, 0), ['emd_cd2', 'emd_nm2']]

for near_emd_cd, near_emd_nm in nears.values:
    geo.loc[geo['emd_cd'] == near_emd_cd, 'value'] = 100

geo.loc[geo['emd_cd'] == target_cd, 'value'] = 200

print('target:', target_cd, target_nm)
print('nears:', list(map(lambda x: x[1], nears.values)))

show_geo(geo, column='value')